In [1]:
from custom_modules import LicenseDatabase
import sys
import replicate
from dotenv import load_dotenv
import os

sys.path.append("..")
load_dotenv()
REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")
os.environ['REPLICATE_API_TOKEN'] = REPLICATE_API_TOKEN
api_endpoint = "https://replicate.com/yorickvp/llava-13b/api?tab=python"
headers = {"Content-Type": "application/json"}
db = LicenseDatabase("../data/storage/license_database.db")
states_dir = "../data/images/states/"

def extract_text(image_path):
    output = replicate.run(
        "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
        input={
            "image": open(image_path, "rb"),
            "prompt": """Use the text from the image to fill in the values for each category within the JSON Schemas.
            
                If a category is not present, write the value as "None"
                If a category is present but you cannot extract the value, write the value as "Unknown Value"
                
                Remember to look for non-character symbols that can indicate organ donor or veteran status.
                veteran status is often indicated by looking for the word 'Veteran' in an unexpected place,
                as a unique style font, not designated with any label.
                A circle that contains a star indicates that it is `'real_id': 'true'`.
                Replace the "string", "boolean" and "integer" values with the extracted text.
                
                ```
                {\
                    "name": {\
                        "first": "string",\
                        "middle": "string",\
                        "last": "string"\
                            },\
                    "address": {\
                        "street": "string",\
                        "city": "string",\
                        "state": "string",\
                        "zip": "integer"\
                            },\
                    "real_id": "boolean",\
                    "organ_donor": "boolean",\
                    "veteran_status": "boolean",\
                    "DD": "integer",\
                    "customer_number": "string",\
                    "id_type": "string",\
                    "issue_date": "string",\
                    "expiration_date": "string",\
                    "date_of_birth": "string",\
                    "sex": "string",\
                    "eyes": "string",\
                    "class": "string",\
                    "height": "float",\
                    "endorsements": "string",\
                    "restrictions": "string",\
                    "signature": "boolean"\
                        }"""
        }
    )
    extracted_text = ""
    for item in output:
        extracted_text += item
        print(item, end="")
    return extracted_text


def select_state_directory(state_name):
    for dir_name in os.listdir(states_dir):
        dir_path = os.path.join(states_dir, dir_name)
        if dir_name.lower().startswith(state_name.lower()):
            return dir_path


def process_input(input_path):
    if os.path.isfile(input_path):
        if input_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            extracted_data = extract_text(input_path)
            filename = os.path.basename(input_path)
            db.insert_license_data(filename, extracted_data, input_path)

    elif os.path.isdir(input_path):
        for filename in os.listdir(directory_path):
            if filename.endswith(".png") or filename.endswith("jpg") or filename.endswith("jpeg"):
                file_path = os.path.join(directory_path, filename)
                extracted_data = extract_text(file_path)
                print(filename.upper(), " : ", extracted_data)
                db.insert_license_data(filename, extracted_data, file_path)
    else:
        print(f"'{input_path}' is an INVALID INPUT. it is neither a path to a valid image file or a directory of valid images")


directory_path = select_state_directory('new jersey')
process_input(directory_path)
print("UPDATED_DataBase:: ", db)


ModuleNotFoundError: No module named 'custom_modules'

In [ ]:
%cd ../demos/
!streamlit run database_viewer.py


In [ ]:
# db.insert_license_data(result)
# customer_number = result.get('customer_number', {}).get('type')
# record = db.retrieve_license_data(customer_number)
# updated_data = result.copy()
# db.update_license_data(customer_number, updated_data)

In [ ]:
# images_dir = "../data/images/"
# alabama_folder = "../data/images/Alabama drivers license/"
# alabama_image = "../data/images/states/Alabama drivers license/original_635733316643777774-aladriverlicense.jpg"
# new_mexico_folder = "../data/images/states/New Mexico drivers license/"
# new_mexico_image = "../data/images/states/New Mexico drivers license/original_Buy-New-Mexico-driver-license-online.jpg"
# colorado_image = "../data/images/states/Colorado drivers license/original_colorado.sb251.drivers.license.jpg"


In [ ]:
# # only  converted so that it actually displays
# sample_image = display.Image(filename=colorado_image)
# sample_image


In [ ]:
# def test(image_path):
#     output = replicate.run(
#         "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
#         input={
#             "image": open(image_path, 'rb'),
#             "prompt": """Act as an OCR and EXTRACT all of the TEXT from the image. Label and organize your response in a json format"""
#         }
#     )
#     extracted_text = ""
#     for item in output:
#         extracted_text += item
#         print(item, end="")
#     return extracted_text
# # test_text = test(colorado_image)

In [ ]:
# def test_model(image_path):
#     output = replicate.run(
#         "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
#         input={
#             "image": open(image_path, 'rb'),
#             "prompt": """Act as an OCR and extract the text from the image, returned in a valid json format.
#             The labels in the image you see may not match the labels in the example, however, conceptually they are the same.
#             For each label, provide the value found within the image.
#             In other words, replace the values of "string", "integer" and "boolean" with the appropriate text from the image.
#             Return the structure below, filled out with the values from above.
#             Lastly, pay close attention to the organ_donor, real_id and veteran_status indicators.
#             Here is an example that describes the format:
#             {\
#                     "name": {\
#                         "first": "string",\
#                         "middle": "string",\
#                         "last": "string"\
#                             },\
#                     "address": {\
#                         "street": "string",\
#                         "city": "string",\
#                         "state": "string",\
#                         "zip": "integer"\
#                             },\
#                     "real_id": "boolean",\
#                     "organ_donor": "boolean",\
#                     "veteran_status": "boolean",\
#                     "DD": "integer",\
#                     "customer_number": "string",\
#                     "id_type": "string",\
#                     "issue_date": "string",\
#                     "expiration_date": "string",\
#                     "date_of_birth": "string",\
#                     "sex": "string",\
#                     "eyes": "string",\
#                     "class": "string",\
#                     "height": "float",\
#                     "endorsements": "string",\
#                     "restrictions": "string",\
#                     "signature": "boolean"\
#                         }
#             """
#         }
#     )
#     extracted_text = ""
#     for item in output:
#         extracted_text += item
#         print(item, end="")
#     return extracted_text


In [ ]:
# result = extract_text(new_mexico_image)
# test_result = test_model(colorado_image)

#### Here is an example of an address city change

```python
updated_data["address"]["city"]["type"] = "NewCity"
```